In [1]:
import _09_utility as util

dataset = util.load_large_csv('spx_features.csv')
fssr = util.fn_all_subsets(True,0,True)
fssd = util.fnsubset_to_indexdictlist(dataset.columns,fssr)
fss = []
for d in fssd:
    fss.append(list(d.keys()))
print(fss)
dataset = dataset.values
arr_low = dataset[:,1]
arr_close = dataset[:,2]

Trying to load CSV file into DataFrame...
loaded chunk 1 of size: 103800164 -> 57475164
loaded chunk 2 of size: 103800164 -> 57475164
loaded chunk 3 of size: 103800164 -> 57475164
loaded chunk 4 of size: 103800164 -> 57475164
loaded chunk 5 of size: 103800164 -> 57475164
loaded chunk 6 of size: 103800164 -> 57475164
loaded chunk 7 of size: 103800164 -> 57475164
loaded chunk 8 of size: 103800164 -> 57475164
loaded chunk 9 of size: 103800164 -> 57475164
loaded chunk 10 of size: 103800164 -> 57475164
loaded chunk 11 of size: 103800164 -> 57375164
loaded chunk 12 of size: 103800164 -> 57475164
loaded chunk 13 of size: 103800164 -> 57475164
loaded chunk 14 of size: 92415380 -> 51171306
concat chunks
concatted chunks
Success.
Size of dataset:	798346306
[[0, 1, 2], [3], [151, 152, 153], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,

In [2]:
log_normalize = True
lag_allowance = 30
generations = 10
hold_for = 15
log_normalize = True
lag_allowance = 30
generations = 10
num_eras = 1
hold_for = 15
criteria = 'r2'
elite_criteria = 1
num_parents = 10
rep_mode = 'exponential'
part_mproba = 0.000
ptrn_mproba = 0.000
using_batches = False
recomp_loaded_batches = []

In [3]:
import warnings
warnings.simplefilter("error", category=RuntimeWarning)

import _00_gene as _0
import _01_initialization as _1
import _02_evaluation as _2
import _05_reproduction as _5
import _10_era as _10

In [4]:
import _00_gene as _0
load_genes = _0.load_custom_genes(
	fss=fss,lag_allowance=lag_allowance,gene_pattern_sets=[[#0
        (412, 10, 'gt', 412, 14),(454, 8, 'gt', 464, 24)],[#1
		(45, 1, 'gt', 46, 7),(6, 24, 'gt', 6, 16)],[#2
		(199, 16, 'lt', 164, 15),(421, 19, 'lt', 410, 22)],[#3
		(412, 12, 'gt', 408, 19),(454, 8, 'gt', 476, 10)],[#4
		(411, 23, 'gt', 419, 22),(204, 25, 'lt', 154, 21)],[#5
		(411, 4, 'lt', 444, 13),(514, 7, 'lt', 494, 8)],[#6
		(423, 22, 'lt', 423, 14),(428, 5, 'lt', 428, 20)],[#7
		(212, 10, 'lt', 207, 18),(77, 13, 'lt', 68, 4)],[#8
		(440, 27, 'gt', 437, 0),(423, 12, 'lt', 423, 6)],[#9
		(438, 27, 'lt', 435, 17),(403, 6, 'gt', 401, 0)],[#10
		(466, 20, 'gt', 470, 15),(436, 10, 'lt', 440, 29)],[#11
		(514, 2, 'lt', 491, 16),(428, 23, 'gt', 428, 14),(417, 26, 'gt', 419, 17)]])

In [5]:
import _05_reproduction as _5

recomp_loaded = _5.expansive_recomposition(
    population=load_genes,
    num_patterns=3
)

Expanded population: 12 -> 2300 at length 3.


In [6]:
arr_returns, arr_kratio = _1.collect_parallel_metrics(
	arr_close=arr_close,
	arr_low=arr_low,
	hold_for=hold_for,
	lag_allow=lag_allowance,
	log_normalize=log_normalize
)

In [7]:
era_args = {
		'generations'	:	1,
		'dataset'		:	dataset,
		'fss'			:	fss,
		'criteria'	:	criteria,
        'with_array'	:	False,
		'log_normalize'	:	log_normalize,
		'hold_for'	:	hold_for,
		'lag_allowance'	:	lag_allowance,
		'arr_returns'	:	arr_returns,
		'arr_kratio'	:	arr_kratio,
		'elite_criteria' : 1,
		'num_parents' : 4,
		'rep_mode' : 'exponential',
		'part_mproba' : 0.000,
		'ptrn_mproba' : 0.000,
		'use_strict_filter'	:	True,
		'strict_filter_kwargs'	:	{
			'avg_return':(2.50 / 25000),
			'tot_return':0.25,
			'profit_factor':0.5,
			'kelsch_ratio':2.0,
			'entry_frequency':0.005,
			'r2':0.8
		}}

In [8]:
import _05_reproduction as _5

recomp_loaded = _5.expansive_recomposition(
    population=load_genes,
    num_patterns=3
)

Expanded population: 12 -> 2300 at length 3.


In [10]:
import _11_eon as _11

endeon = _11.autobatch_eon(
    eon_num=0,
    population=recomp_loaded,
    criteria=criteria,
    filter_tightening={},
    era_kwargs=era_args,
    spec_batch_size=200
)

325000  EON 1 ERA 1 GEN 1 (r2):::: Zero genes survived.
325000  EON 1 ERA 2 GEN 1 (r2):::: AVG 0.93907, BEST 0.94821, FROM 3 GENES.
325000  EON 1 ERA 3 GEN 1 (r2):::: AVG 0.92557, BEST 0.92557, FROM 1 GENES.
325000  EON 1 ERA 4 GEN 1 (r2):::: AVG 0.89871, BEST 0.89871, FROM 1 GENES.
325000  EON 1 ERA 5 GEN 1 (r2):::: AVG 0.89184, BEST 0.9327, FROM 6 GENES.
325000  EON 1 ERA 6 GEN 1 (r2):::: AVG 0.91055, BEST 0.94739, FROM 5 GENES.
325000  EON 1 ERA 7 GEN 1 (r2):

: 

In [ ]:
len(endeon)

In [ ]:
import _10_era as _10
reload(_10)
reload(_2)

endtime = _10.era(
    eon_num=0,
	era_num	=	0,
	new_population	=	endeon,
	generations	=	1,
	dataset		=	dataset,
	fss			=	fss,
	criteria	=	criteria,
    with_array=True,
	log_normalize	=	log_normalize,
	hold_for	=	hold_for,
	lag_allowance	=	lag_allowance,
	arr_returns	=	arr_returns,
	arr_kratio	=	arr_kratio,
	elite_criteria = 1,
	num_parents = 4,
	rep_mode = 'exponential',
	part_mproba = 0.000,
	ptrn_mproba = 0.000,
	use_strict_filter	=	True,
	strict_filter_kwargs	=	{
		'avg_return':(2.50 / 25000),
		'tot_return':0.25,
		'profit_factor':0.5,
		'kelsch_ratio':2.0,
		'entry_frequency':0.01,
           'r2':0.6
}
)

In [ ]:
reload(_2)
print(_2.show_best_gene_patterns(endtime,criteria,fssd))
_2.show_returns(endtime[0]._array_returns, arr_close=arr_close, gene_kwargs={"population":endtime,"criteria":criteria,"fss":fssd})

In [ ]:
from importlib import reload
import _00_gene as _0
import _01_initialization as _1
import _02_evaluation as _2
reload(_0)
reload(_2)

log_normalize = True
lag_allowance = 30
generations = 5
hold_for = 15
criteria = 'mkr'
elite_criteria = 1
num_parents = 10
rep_mode = 'exponential'
part_mproba = 0.000
ptrn_mproba = 0.000

arr_returns, arr_kratio = _1.collect_parallel_metrics(
	arr_close=arr_close,
	arr_low=arr_low,
	hold_for=hold_for,
	lag_allow=lag_allowance,
	log_normalize=log_normalize
)


load_genes = _0.load_custom_genes(
	fss=fssd,
	lag_allowance=lag_allowance,
	gene_pattern_sets=[
	[#gene 0
		(412, 10, 'gt', 412, 14),
		(454, 8, 'gt', 464, 24)
	],[#gene 6
		(423, 22, 'lt', 423, 14),
        (428, 5, 'lt', 428, 20)
	],[#gene 7
		(212, 10, 'lt', 207, 18),
        (77, 13, 'lt', 68, 4)
	],[#gene 8
		(440, 27, 'gt', 437, 0),
        (423, 12, 'lt', 423, 6)
	],
]
)


new = _2.show_combined_performance(
    population=load_genes,
    arr_close=arr_close,
    arr_low=arr_low,
    arr_kratio=arr_kratio,
    arr_returns=arr_returns,
    data=dataset,
    hold_for=hold_for,
    lag_allow=lag_allowance,
    specific_data=None,
    log_normalize=log_normalize,
    criteria=criteria,
    fss=fssd
)
